In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install selenium

In [ ]:
%cd /content/drive/MyDrive/CHF_Project

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

warnings.filterwarnings('ignore')

df = pd.read_csv("Becker_Tube_Dataset.csv")
columns = ['L(m)', 'D(m)', 'p(Mpa)', 'G(kgm-2s-1)', '∆T_sub(k)', 'q(MWm-2)', 'Z_cr(o/p)']

In [ ]:
train_df = df[columns]
train_df

In [ ]:
from scipy import stats
train_df = train_df[(np.abs(stats.zscore(train_df['Z_cr(o/p)'])) < 3)]
train_df

In [ ]:
train_df.describe()

In [ ]:
############# Standardising the Data ############
# X_real = train_df.to_numpy()
# scaler = StandardScaler()
# X_real = scaler.fit_transform(X_real)
# y_real = np.ones((X_real.shape[0],))
# print(X_real.shape)
# print(y_real.shape)
# Standardised_df = pd.DataFrame(X_real, columns = columns)
# Standardised_df

In [ ]:
############## Min Max Normalising the data #############
X_real = train_df.to_numpy()
scaler = StandardScaler()
X_real = scaler.fit_transform(X_real)
y_real = np.ones((X_real.shape[0],))
print(X_real.shape)
print(y_real.shape)
Standardised_df = pd.DataFrame(X_real, columns = columns)
Standardised_df

In [ ]:
######### GETTING THE GPU ###########
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)



In [ ]:
def get_generator_block(input_dim, output_dim):      #Generator Block
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        #nn.BatchNorm1d(output_dim),   
        nn.ReLU(inplace=True),
    )
class GANs_Generator(nn.Module):     #Generator Model

    def __init__(self, z_dim, im_dim, hidden_dim):
        super(GANs_Generator, self).__init__()
        self.generator = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
            #nn.Sigmoid()
        )
    def forward(self, noise):
        return self.generator(noise)    
   
    def get_generator(self):
        return self.generator

def get_discriminator_block(input_dim, output_dim):       #Discriminator Block
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace=True)        
    )
class GANs_Discriminator(nn.Module):         #Discriminator Model
    def __init__(self, im_dim, hidden_dim):
        super(GANs_Discriminator, self).__init__()
        self.discriminator = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image):
        return self.discriminator(image)
    
    def get_disc(self):
        return self.discriminator

In [ ]:

#This file is used in GANs_fit.py file

class train_discriminator():
  def __init__(self, real_data, latent_data, opt_d, generator, discriminator, device):
    self.real_data = real_data
    self.latent_data = latent_data
    self.opt_d = opt_d
    self.discriminator = discriminator
    self.generator = generator
    self.device = device

  def __call__(self):
    self.opt_d.zero_grad()

    # Pass real data through discriminator
    real_preds = self.discriminator(self.real_data)
    real_targets = torch.ones_like(real_preds, device=self.device)

    criterion = nn.BCEWithLogitsLoss()
    real_loss = criterion(real_preds, real_targets)
    real_score = torch.mean(real_preds).item()
    
    # Generate fake data
    #latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
    fake_data = self.generator(self.latent_data)
    #fake = gen(X_oversampled.float().to(device))

    # Pass fake data through discriminator    
    fake_preds = self.discriminator(fake_data)
    fake_targets = torch.zeros_like(fake_preds, device=self.device)

    
    #fake_loss = F.binary_cross_entropy(fake_preds, fake_targets)

    fake_loss = criterion(fake_preds, fake_targets)
    fake_score = torch.mean(fake_preds).item()

    # Update discriminator weights
    loss = (real_loss + fake_loss)/2
    loss.backward()
    self.opt_d.step()
    return loss.item(), real_score, fake_score


class train_generator():
    def __init__(self, latent_data, opt_g, generator, discriminator, device):
        self.latent_data = latent_data
        self.opt_g = opt_g
        self.generator = generator
        self.discriminator = discriminator
        self.device = device


    def __call__(self):
        # Clear generator gradients
        self.opt_g.zero_grad()
        
        # Generate fake data
        fake_data = self.generator(self.latent_data)
        
        # Try to fool the discriminator
        preds = self.discriminator(fake_data)   #We put the fake data generated by generator into the discriminator
        targets = torch.ones_like(preds, device=self.device)

        criterion = nn.BCEWithLogitsLoss()
        loss = criterion(preds, targets)
        
        # Update generator weights
        loss.backward()
        self.opt_g.step()
        
        return loss.item()

In [ ]:
######################## Wasserstien GANs Loss #######################
class train_Wdiscriminator():
  def __init__(self, real_data, latent_data, opt_d, generator, discriminator, device):
    self.real_data = real_data
    self.latent_data = latent_data
    self.opt_d = opt_d
    self.discriminator = discriminator
    self.generator = generator
    self.device = device

  def __call__(self):
    self.opt_d.zero_grad()

    # Pass real data through discriminator
    real_preds = self.discriminator(self.real_data)
    real_score = torch.mean(real_preds)
    
    # Generate fake data
    fake_data = self.generator(self.latent_data)

    # Pass fake data through discriminator    
    fake_preds = self.discriminator(fake_data)
    fake_score = torch.mean(fake_preds)
    
    # Update discriminator weights
    loss = -(torch.mean(real_preds) - torch.mean(fake_preds))
    loss.backward()
    self.opt_d.step()

    #Weight Clipping
    for p in self.discriminator.parameters():
      p.data.clamp_(-0.01, 0.01)

    return loss.item(), real_score, fake_score


class train_Wgenerator():
    def __init__(self, latent_data, opt_g, generator, discriminator, device):
        self.latent_data = latent_data
        self.opt_g = opt_g
        self.generator = generator
        self.discriminator = discriminator
        self.device = device


    def __call__(self):
        # Clear generator gradients
        self.opt_g.zero_grad()
        
        # Generate fake data
        fake_data = self.generator(self.latent_data)
        
        # Try to fool the discriminator
        preds = self.discriminator(fake_data)   #We put the fake data generated by generator into the discriminator

        # Update generator weights
        loss = -torch.mean(preds)
        loss.backward()
        self.opt_g.step()
        
        return loss.item()

In [ ]:
device = get_default_device()

In [ ]:
#lr = 0.0002
epochs = 100
batch_size = 32

In [ ]:
##### Wrapping all the tensors in a Tensor Dataset. #####
my_dataset = TensorDataset(torch.Tensor(X_real) ,torch.Tensor(y_real))

##### Loading our Tensor Dataset into a Dataloader. #####
train_dl = DataLoader(my_dataset, batch_size=batch_size, shuffle=True) 
train_dl = DeviceDataLoader(train_dl, device)

##### Initialising the generator and discriminator objects ######
gen = GANs_Generator(X_real.shape[1], X_real.shape[1], 128)   #z_dim = 7, im_dim = 7, hidden_dim = 128
disc = GANs_Discriminator(X_real.shape[1], 128)

##### Loading the model in GPU #####
generator = to_device(gen.generator, device)      
discriminator = to_device(disc.discriminator, device)

In [ ]:
# ############################## Vanilla GANs ##################################
# torch.cuda.empty_cache()
    
# # Losses & scores
# losses_g = []
# losses_d = []
# real_scores = []
# fake_scores = []

# lr = 0.0002
# # Create optimizers
# opt_d = torch.optim.Adam(discriminator.parameters(), lr= lr, betas=(0.5, 0.999))
# opt_g = torch.optim.Adam(generator.parameters(), lr= lr, betas=(0.5, 0.999))

# for epoch in range(epochs):
#   for real_data, _ in tqdm(train_dl):
#       # Train discriminator
#       latent_data = torch.randn(real_data.shape[0], real_data.shape[1], device=device)
#       train_disc = train_discriminator(real_data, latent_data, opt_d, generator, discriminator, device)
#       loss_d, real_score, fake_score = train_disc()
#       # Train generator
#       train_gen = train_generator(latent_data, opt_g, generator, discriminator, device)
#       loss_g = train_gen()
      
#   # Record losses & scores
#   losses_g.append(loss_g)
#   losses_d.append(loss_d)
#   real_scores.append(real_score)
#   fake_scores.append(fake_score)

#   # Log losses & scores (last batch)
#   print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
#       epoch+1, epochs, loss_g, loss_d, real_score, fake_score))

In [ ]:
############################## Wasserstein GANs ##################################
torch.cuda.empty_cache()
    
# Losses & scores
losses_g = []
losses_d = []
real_scores = []
fake_scores = []
lr = 0.00005

GANs_noise = torch.randn((X_real.shape[0]), (X_real.shape[1]), device=device)
# for lr in lr:
# Create optimizers
opt_d = torch.optim.RMSprop(discriminator.parameters(), lr= lr)
opt_g = torch.optim.RMSprop(generator.parameters(), lr= lr)

for epoch in range(epochs):
  for real_data, _ in tqdm(train_dl):
      # Train discriminator
      latent_data = torch.randn(real_data.shape[0], real_data.shape[1], device=device)
      train_disc = train_Wdiscriminator(real_data, latent_data, opt_d, generator, discriminator, device)
      loss_d, real_score, fake_score = train_disc()
      # Train generator
      train_gen = train_Wgenerator(latent_data, opt_g, generator, discriminator, device)
      loss_g = train_gen()
      
  # Record losses & scores
  losses_g.append(loss_g)
  losses_d.append(loss_d)
  real_scores.append(real_score)
  fake_scores.append(fake_score)

  # Log losses & scores (last batch)
  print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
      epoch+1, epochs, loss_g, loss_d, real_score, fake_score))
  # Trained_X_oversampled = generator(GANs_noise.float().to(device)).cpu().detach().numpy()
  # oversampled_df = pd.DataFrame(Trained_X_oversampled, columns = columns)
  # Oversampled_real_life_df = scaler.inverse_transform(oversampled_df)
  # Oversampled_real_life_df = pd.DataFrame(Oversampled_real_life_df, columns = columns)
  # print(Oversampled_real_life_df)
  # print(Oversampled_real_life_df.describe())
  # Oversampled_real_life_df.to_csv("Generated_data_reallike_lr_{}.csv".format(lr))




  0%|          | 0/14 [00:00<?, ?it/s]

Epoch [1/100], loss_g: -0.0106, loss_d: -0.0000, real_score: 0.0106, fake_score: 0.0106


  0%|          | 0/14 [00:00<?, ?it/s]

Epoch [2/100], loss_g: -0.0105, loss_d: -0.0002, real_score: 0.0107, fake_score: 0.0105


  0%|          | 0/14 [00:00<?, ?it/s]

...

Epoch [98/100], loss_g: -0.0085, loss_d: 0.0048, real_score: 0.0054, fake_score: 0.0102


  0%|          | 0/14 [00:00<?, ?it/s]

Epoch [99/100], loss_g: -0.0018, loss_d: -0.0083, real_score: 0.0113, fake_score: 0.0030


  0%|          | 0/14 [00:00<?, ?it/s]

Epoch [100/100], loss_g: -0.0009, loss_d: -0.0036, real_score: 0.0057, fake_score: 0.0022


In [ ]:
GANs_noise = torch.randn((X_real.shape[0]), (X_real.shape[1]), device=device)
Trained_X_oversampled = generator(GANs_noise.float().to(device)).cpu().detach().numpy()

In [ ]:
oversampled_df = pd.DataFrame(Trained_X_oversampled, columns = columns)
oversampled_df

,L(m),D(m),p(Mpa),G(kgm-2s-1),∆T_sub(k),q(MWm-2),Z_cr(o/p)
0,-0.079930,-0.403495,-1.146010,2.382150,-0.032151,2.516660,2.199348
1,-0.082692,-0.124665,-0.616934,1.191579,0.181127,1.363127,1.036881
2,0.005866,-0.019815,-0.630752,1.236737,0.308344,1.549680,1.132559
3,-0.032868,-0.360830,-0.875426,1.767244,0.096049,1.948400,1.602482
4,-0.138602,-0.294450,-0.799824,1.577265,0.141863,1.716857,1.380789
...,...,...,...,...,...,...,...
415,0.295738,0.177205,-1.455627,2.632211,0.543866,3.547098,2.471875
416,0.200688,-0.123079,-0.830569,1.604113,0.293816,2.128340,1.538039
417,-0.050421,-0.185700,-0.544099,1.040062,0.116228,1.258782,0.971941
418,0.011345,-0.187224,-0.715987,1.303076,0.202657,1.590689,1.219422


In [ ]:
Oversampled_real_life_df = scaler.inverse_transform(oversampled_df)
Oversampled_real_life_df = pd.DataFrame(Oversampled_real_life_df, columns = columns)
Oversampled_real_life_df

,L(m),D(m),p(Mpa),G(kgm-2s-1),∆T_sub(k),q(MWm-2),Z_cr(o/p)
0,7.006755,0.013064,2.772252,3731.512451,10.405853,1.361577,7.142600
1,7.006676,0.014123,7.623547,2762.664062,10.742932,1.056827,5.655646
2,7.009216,0.014521,7.496846,2799.412354,10.943995,1.106112,5.778031
3,7.008105,0.013226,5.253339,3231.121582,10.608470,1.211450,6.379127
4,7.005072,0.013478,5.946564,3076.523193,10.680877,1.150279,6.095551
...,...,...,...,...,...,...,...
415,7.017531,0.015270,-0.066739,3935.004150,11.316231,1.633806,7.491199
416,7.014805,0.014129,5.664649,3098.370605,10.921034,1.258987,6.296696
417,7.007601,0.013891,8.291403,2639.364502,10.640362,1.029261,5.572579
418,7.009373,0.013886,6.715298,2853.396973,10.776960,1.116947,5.889141


In [ ]:
Oversampled_real_life_df.describe()

,L(m),D(m),p(Mpa),G(kgm-2s-1),∆T_sub(k),q(MWm-2),Z_cr(o/p)
count,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000
mean,7.009349,0.013765,5.110992,3206.130859,10.871475,1.239012,6.348017
std,0.003394,0.000461,1.854887,317.994690,0.286420,0.128872,0.469967
min,7.002828,0.012311,-2.592736,2505.071045,10.273915,0.954165,5.299215
25%,7.006883,0.013475,4.037783,2984.274353,10.654954,1.148679,6.015500
50%,7.008832,0.013803,5.307981,3192.492432,10.860830,1.226602,6.297698
75%,7.011174,0.014055,6.398640,3392.962769,11.059959,1.313949,6.605475
max,7.022538,0.015270,9.136522,4506.293457,11.782912,1.765459,8.214943


In [ ]:
train_df.describe()


,L(m),D(m),p(Mpa),G(kgm-2s-1),∆T_sub(k),q(MWm-2),Z_cr(o/p)
count,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000,420.000000
mean,7.009048,0.014597,13.280452,1792.995238,10.456667,0.696706,4.329333
std,0.028720,0.003803,9.180314,814.738452,1.582354,0.264503,1.280662
min,7.000000,0.010000,2.980000,496.300000,4.600000,0.146000,0.630000
25%,7.000000,0.014900,7.035000,1009.150000,9.800000,0.499000,3.387500
50%,7.000000,0.014900,13.990000,1968.800000,10.300000,0.706000,4.425000
75%,7.000000,0.014900,17.980000,2527.150000,11.000000,0.873500,5.350000
max,7.100000,0.024700,119.800000,3493.100000,20.700000,1.270000,7.000000


In [ ]:
#oversampled_df.to_csv("Generated_data_Epochs_{}_BatchSize_{}.csv".format(epochs, batch_size))
name = "Generated_data_e{}_bs{}_1.csv".format(epochs, batch_size)
Oversampled_real_life_df.to_csv(name)

Generated_data_e100_bs32_1.csv


In [ ]:
data_df = pd.read_csv(name, usecols = ['p(Mpa)'])
data_df.head()

In [ ]:
out_df = pd.DataFrame()

In [ ]:
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

In [ ]:
# driver.get("https://www.peacesoftware.de/einigewerte/wasser_dampf_e.html")
# driver.maximize_window()
for val in data_df.iterrows():
    pressure = val[1].values[0]
    from selenium import webdriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',options=options)
    #driver = webdriver.Chrome('C:\chromedriver')
    driver.get("https://www.peacesoftware.de/einigewerte/wasser_dampf_e.html")
    driver.maximize_window()
 
    driver.find_elements(By.NAME, "druckunit")[-1].send_keys(Keys.DOWN, Keys.DOWN, Keys.ENTER)
    driver.find_elements(By.NAME, "druck")[-1].send_keys(pressure, Keys.ENTER)
    table = driver.find_elements('xpath', '/html/body/table[2]/tbody/tr')

    write_dict = {}
    for i in range(2, 25):
        row = []
        for j in range(1, 4):
            d = driver.find_element('xpath', "/html/body/table[2]/tbody/tr[{}]/td[{}]".format(i,j)).text
            row.append(d)
        write_dict[row[0] + ' ' + row[-1]] = row[1]

    out_df = pd.concat([out_df, pd.Series(write_dict).to_frame().T], ignore_index=True)
    driver.quit()

In [ ]:
name1 = "Scraped" + name
out_df.to_csv(name1, index = False)